In [1]:
import cv2
import numpy as np
import os
from PIL import ImageGrab, Image
import os.path as osp
from ultralytics import YOLO
import pydirectinput
import time
# load model
model = YOLO("runs/detect/train18/weights/best.pt")

# 화면 크기 설정
screen_size = (1280, 720)
# 중앙 300x300 영역 설정
center_size=300
center_x = (screen_size[0] - center_size) // 2
center_y = (screen_size[1] - center_size) // 2
bbox = (center_x, center_y, center_x + center_size, center_y + center_size)
#저장 경로 디폴트셋팅
train_path = "images/"
label_path = "labels/"

# 이미지 저장할 폴더 생성
if not osp.isdir(train_path):
    os.makedirs(train_path)

# 텍스트 파일 저장할 폴더 생성
if not osp.isdir(label_path):
    os.makedirs(label_path)

녹화

In [3]:
# 녹화할 파일명과 코덱 설정
output_file = 'center_screen_recording.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# 비디오 녹화 객체 생성
out = cv2.VideoWriter(output_file, fourcc, 20.0, (center_size, center_size))
while True:
    # 현재 화면 캡쳐
    img = ImageGrab.grab(bbox=bbox)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # 화면에 표시 및 파일에 저장
    img=np.array(img)
    cv2.imshow('Center Screen Recording', img)
    out.write(img)
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) == ord('q'):
        break
# 사용한 자원 해제
out.release()
cv2.destroyAllWindows()

In [ ]:
heroes_num = {'mauga': '1' , 'ana' : '2', 'ashe' : '3', 'baptiste' : '4', 'bastion' : '5', 'brigitte' : '6', 'cassidy' : '7',
                  'd.va' : '8', 'doomfist' : '9', 'echo' : '10', 'genji' : '11', 'hanzo' : '12', 'illari' : '13', 'junkerqeen' : '14',
                  'junkrat' : '15', 'kiriko' : '16', 'lifeweaver' : '17', 'lucio' : '18', 'mei' : '19', 'mercy' : '20', 'moira' : '21',
                  'orisa' : '22', 'pharah' : '23', 'ramattra' : '24', 'reaper' : '25', 'reinhardt' : '26', 'roadhog' : '27', 'sigma' : '28',
                  'sojourn' : '29', 'soldier:76' : '30', 'sombra' : '31', 'symmetra' : '32', 'torbjorn' : '33', 'tracer' : '34',
                  'widowmaker' : '35', 'winston' : '36', 'wreckingball' : '37', 'zarya' : '38', 'zenyatta' : '39'} 

# 영웅 이름 리턴하는 함수
def get_hero_num(key):
    global heroes_num
    if key in heroes_num:
        return heroes_num[key]
    else:
        return "해당하는 영웅을 찾을 수 없습니다."

In [4]:
# 박스의 w/2,h/2 는 size의 크기와 같도록 함
size=5

# 폴더 내 파일 개수 확인
def file_count(path):
    global count 
    folder_path = path
    file_list = os.listdir(folder_path)
    count = len(file_list)
    return count
count = file_count(train_path)

# 클릭 좌표 관련 변수
point=[]
click_coordinate = []
pointed=False
while_rec=False

# 캐릭터별 번호
cls={"봇": 0,"트레": 1}

# 마우스 이벤트
def mouse_callback(event, x, y, flags, param):
    global point, img, count, pointed, while_rec
    global size

    if event == cv2.EVENT_LBUTTONDOWN:
        point = [x,y]
        tempimg=img.copy()

        if while_rec==True:
            pointed=True
            cv2.putText(tempimg, f"stop count:{count}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
            cv2.rectangle(tempimg, (point[0]-size, point[1]-size), (point[0]+size, point[1]+size), (0, 0, 255), 1)
            cv2.imshow("win",tempimg)
    
    # 마우스 휠에 따라 사각형 크기 조절
    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            size += 1
        elif size > 1:
            size -= 1

# 마우스 클릭 좌표 기준 사각형 그리기
def make_rectangle():
    global img, pointed
    global point, while_rec, count
    while_rec=True

    key=cv2.waitKey(0)

    # 스페이스 입력 시 영상 정지 및 마우스 클릭 좌표 정보를 텍스트 파일에 저장
    if key == ord(" "):
        if pointed==True:
            count = file_count()
            
            # 텍스트 파일 열기
            text = open_file(label_path + str(count) + ".txt", "w")

            print(f"{point[0]},{point[1]}")

            # 텍스트 파일에 좌표값 저장
            text.write(f"{cls['봇']} {float(point[0])/center_size} {float(point[1])/center_size} {float(size)*2/center_size} {float(size)*2/center_size}")

            # faces/count.jpg 로 저장
            cv2.imwrite(train_path + str(count) + '.jpg', img)
            
            # 텍스트 파일 닫기
            text.close()
            pointed=False
    while_rec=False

# 파일 열기 함수
def open_file(filename, mode):
    try:
        return open(filename, mode)
    except FileNotFoundError:
        return open(filename, "w")

D: 60프레임 이후로 스킵
A: 60프레임 이전으로 스킵
Space: 화면 정지
좌클릭: 중앙점 생성, 정사각형 생성
횔업&다운: 정사각형 크기 늘리고 줄이고
E&R: 사각형 폭 늘리고 줄이기

In [3]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
cv2.setMouseCallback("win", mouse_callback)
ret,img=readvideo.read()

while ret:
    key = cv2.waitKey(33)

    # 프레임 조정
    if key == ord("d"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) + 60)
    elif key == ord("a"):
        readvideo.set(cv2.CAP_PROP_POS_FRAMES, readvideo.get(cv2.CAP_PROP_POS_FRAMES) - 60)

    # 사각형 생성
    elif key == ord(" "):
        make_rectangle()

    # 영상 종료
    elif key == ord("q"):
        break
    
    ret,img=readvideo.read()

    cv2.imshow("win", img)

readvideo.release()
cv2.destroyAllWindows()

212,133
56,182
129,71


In [15]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
ret,img=readvideo.read()

while ret:
    key = cv2.waitKey(1)

    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
            result = r.boxes.xywh[0].cpu().numpy()
            result = result.tolist()
            result = [round(element) for element in result]
            print(result)
    
            x, y, w, h = result
            cv2.rectangle(img, (x - w, y - h), (x + w, y + h), (0, 0, 255), 2)        
     

    cv2.imshow("win", img)

    ret,img=readvideo.read()

    # 영상 종료
    if key == ord("q"):
        break
readvideo.release()
cv2.destroyAllWindows()


0: 320x320 1 bot, 14.7ms
Speed: 12.0ms preprocess, 14.7ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 13.0ms
Speed: 2.1ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 13.2ms
Speed: 2.0ms preprocess, 13.2ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 1 bot, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 320)
[23, 283, 45, 35]

0: 320x320 

In [3]:
results=model("./images/248.JPG")

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    bb=boxes.xywh[0].cpu().numpy()
    bb=bb.tolist()
    print(bb)
    masks = result.masks  # Masks object for segmentation masks outputs
    print(masks)
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


image 1/1 c:\Users\Administator\Desktop\PJT_MCL\images\248.JPG: 320x320 1 bot, 5.6ms
Speed: 2.0ms preprocess, 5.6ms inference, 6.6ms postprocess per image at shape (1, 3, 320, 320)
[126.93994140625, 74.19407653808594, 34.51824188232422, 33.23259735107422]
None


In [35]:
'''
Overwatch aim bot program
This code runs in the background of your computer as you play Overwatch.
It identifies the location of enemies through image analysis.
Specifically, in Overwatch, each enemy is outlined in Red.
This program takes a screenshot, identifies red pixels on the screen, and moves
your cursor to the center of the red pixels.
I have not searched for the best thesholds to use to identify the red lines and
just the red lines
If this script is run by itself, the Main function is run using these values
key = 0x42 #b key
xbbox = 200
ybbox = 200
FPS=30
AutoAdjust=True
Debug=False
Function ScreenGrab(sct,bbox) gets the screenshot
Function GetOutline(im,bbox) get the red lines
Function PrintScreenGrab(im) and PrintOutline(im,xloc,yloc,bbox) are uses when
debugging is on, to see what is being captured and identified as a red line
Function CursorSnap(xloc,yloc,bbox) moves the cursor based on the red line information
Function Main has an infinite loop, and listens for a key to run the above functions
Virtual key codes can be found by googling "windows Virtual-Key Codes"
For this code to work, your computer must be beefy enough to run overwatch at a
fast frame rate, to reduce input lag. Otherwise, this program will aim
behind your target.
###
Code Written by:
Kyle Shepherd
kyleanthonyshepherd@gmail.com
Sep 23, 2016
###
'''
#### Import BLock ####
# the import block imports needed modules, and spits out a json file with
# version numbers so the code can be repeatable
file = open("ModuleVersions.json", 'w')
modules = {}
import os
import time
import sys
modules['Python'] = dict([('version', sys.version_info)])
import json
modules['json'] = dict([('version', json.__version__)])
import numpy
modules['numpy'] = dict([('version', numpy.__version__)])
import PIL
modules['PIL'] = dict([('version', PIL.__version__)])
from PIL import Image
import mss
modules['mss'] = dict([('version', mss.__version__)])
from mss.windows import MSS as mss
import ctypes
modules['ctypes'] = dict([('version', ctypes.__version__)])
json.dump(modules, file, indent=4, sort_keys=True)
file.close()
#### END Import Block ####
def ScreenGrab(bbox):
    '''
This function captures the current monitor screen using mss
Inputs:
###
bbox: the area of the screen to capture
format: dictionary {'width':,'left':,'height':,'top':}
Example:
bbox={'width':100,'left':100,'height':100,'top':100}
###
It returns an Pil Image object
    '''
    im=mss().grab(bbox)
    im=Image.frombytes("RGB", im.size, im.bgra, "raw", "BGRX")
    return im
def GetOutline(im,bbox):
    '''
This function finds the pixels that contain the red outline
Using HSV pixel values, hue and saturation threshholds are set
Inputs:
###
im: the image to analyze
Format: Pil Image object
bbox: the area of the screen to capture
format: dictionary {'width':,'left':,'height':,'top':}
Example:
bbox={'width':100,'left':100,'height':100,'top':100}
###
It returns a tuple containing (xloc,yloc,OutlineFound)
xloc=x coordinates of the pixels with a red outline, list
yloc=y coordinates of the pixels with a red outline, list
OutlineFound=bool if any red outline was found
    '''
    color=list(im.getdata().convert('HSV')) #convert to HSV for cleaner thresholds
    xbbox=bbox['width']/2
    xloc = []
    yloc = []
    for i in range(0,len(color)):
        if color[i][0] < 8  and color[i][1]>135: # thesholds for red line
            xloc.append(i % (xbbox*2))
            yloc.append(i // (xbbox*2))
    if len(xloc)!=0: # checks to see if red line was found
        OutlineFound=True
    else:
        OutlineFound=False
    return (xloc,yloc,OutlineFound)
def PrintScreenGrab(im):
    '''
This function displays the captured area of the screen using PIL image show
Inputs:
###
im: the image to analyze
Format: Pil Image object
###
    '''
    im.show()
def PrintOutline(im,xloc,yloc,bbox):
    '''
This function displays where the red outline was detected.
Inputs:
###
im: the image to analyze
Format: Pil Image object
xloc: the x coordinates of the pixels containing the red outline
Format: List
yloc: the x coordinates of the pixels containing the red outline
Format: List
bbox: the area of the screen to capture
format: dictionary {'width':,'left':,'height':,'top':}
Example:
bbox={'width':100,'left':100,'height':100,'top':100}
###
    '''
    color=list(im.getdata().convert('HSV'))
    xbbox=int(bbox['width']/2)
    for i in range(0,len(color)):
        im.putpixel((i % (xbbox*2),i // (xbbox*2)),(0, 0, 0)) # makes image black
    for i in range(0,len(xloc)):
        im.putpixel((int(xloc[i]),int(yloc[i])),(255, 0, 0)) # makes pixels red where line exists
    im.show()
def CursorSnap(xloc,yloc,bbox):
    '''
This function performes a relative mouse movement based on the location of the
red outline.
It finds the average x and y coordinate, and moves the mouse there
Inputs:
###
xloc: the x coordinates of the pixels containing the red outline
Format: List
yloc: the x coordinates of the pixels containing the red outline
Format: List
bbox: the area of the screen to capture
format: dictionary {'width':,'left':,'height':,'top':}
Example:
bbox={'width':100,'left':100,'height':100,'top':100}
###
    '''
    xavg=sum(xloc)/len(xloc)
    yavg=sum(yloc)/len(yloc)
    # does the acutal cursor move
    ctypes.windll.user32.mouse_event(1, int(xavg-bbox['width']/2), int(yavg-bbox['height']/2), 0,0)
def Main(key,xbbox,ybbox,FPS=30,AutoAdjust=True,Debug=False):
    '''
This function will run in a loop.
It checks for when the given key is pressed. When the key is pressed, the AimBot
program is activated, and the above functions are called to take a screenshot
and find the enemy
Inputs:
###
key: the virtural key code of the keyboard key to activate the aimbot
Format: windows virtual key code
Example: 0x42
for the b key
xbbox: the width of the screenshot to take
Format: integer
Example: 150
larger values require more processing time, and may cause two enemies to be in
view at the same time.
ybbox: the height of the screenshot to take
Format: integer
Example: 150
larger values require more processing time, and may cause two enemies to be in
view at the same time.
FPS: the Frames Per Second that Overwatch runs on your system
When AutoAdjust is turned on, the size of the screenshot is dynamically adjusted
so that the image processing time is the same length of time as a frame
Format: integer
Example: 30
AutoAdjust: switch to turn on autoadjusting frame size
Format: bool
Example: True
Debug: Switch to output images of the captured screenshot
Format: bool
Example: False
###
    '''
    sct = mss()
    mon=sct.monitors #gets monitor size, so the center can be found
    bbox={'width':int(xbbox*2),'left':int(mon[0]['width']/2-xbbox),'height':int(ybbox*2),'top':int(mon[0]['height']/2-ybbox)}
    while 1==1: # infinite loop
        if ctypes.windll.user32.GetKeyState(key)>1: # listens for key
            t=time.time() # for timing the loop, for autoadjust
            im=ScreenGrab(bbox) # calls ScreenGrab
            (xloc,yloc,OutlineFound)=GetOutline(im,bbox) # calls GetOutline
            if Debug:
                PrintScreenGrab(im)
                PrintOutline(im,xloc,yloc,bbox)
            if OutlineFound:
                CursorSnap(xloc,yloc,bbox)
            if AutoAdjust:
                dTtime=1/FPS-(time.time()-t)
                xbbox = xbbox+numpy.sign(dTtime)*10 # adjusts the frame size by 10 pixels at a time
                ybbox = ybbox+numpy.sign(dTtime)*10
                xbbox=max([xbbox,50]) # floors the frame size, so negatives do not occur
                ybbox=max([ybbox,50])
                bbox={'width':int(xbbox*2),'left':int(mon[0]['width']/2-xbbox),'height':int(ybbox*2),'top':int(mon[0]['height']/2-ybbox)}
            else:
                time.sleep(1/FPS-(time.time()-t)) # makes sure each loop is as long as a frame
if __name__ == "__main__":
    key = 0x42 #b key
    xbbox = 200
    ybbox = 200
    FPS=30
    AutoAdjust=True
    Debug=False
    Main(key,xbbox,ybbox,FPS=FPS,AutoAdjust=AutoAdjust,Debug=Debug)

KeyboardInterrupt: 

프레임마다 중점으로 이동

In [ ]:
readvideo=cv2.VideoCapture("center_screen_recording.avi")
cv2.namedWindow('win')
ret,img=readvideo.read()

while ret:
    key = cv2.waitKey(1)

    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
            result = r.boxes.xywh[0].cpu().numpy()
            result = result.tolist()
            result = [round(element) for element in result]
            print(result)
    
            x, y, w, h = result
            cv2.rectangle(img, (x - w, y - h), (x + w, y + h), (0, 0, 255), 2)        
     

    cv2.imshow("win", img)

    ret,img=readvideo.read()

    # 영상 종료
    if key == ord("q"):
        break
readvideo.release()
cv2.destroyAllWindows()

In [2]:
import pyautogui


# 비디오 캡쳐
while True:
    # 현재 화면 캡쳐
    img = ImageGrab.grab(bbox=bbox)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # 화면에 표시 및 파일에 저장
    img=np.array(img)

    results=model(img)
    for r in results:
        if len(r.boxes.xywh)>0:
            result = r.boxes.xywh[0].cpu().numpy()
            result = result.tolist()
            result = [round(element) for element in result]
            x, y, w, h = result
            # 현재 마우스 위치 얻기
            current_x, current_y = pydirectinput.position()
            print(f"{x} , {y}, {current_x}, {current_y}")
            cv2.rectangle(img, (x - w, y - h), (x + w, y + h), (0, 0, 255), 2)
            
            # 상대 좌표로 마우스 이동
            # pydirectinput.moveTo(current_x-(150-x), current_y-(150-y))
            # pydirectinput.move(int((current_x-(150-x))/10), int((current_y-(150-y))/10))
            #pydirectinput.moveRel(640-(current_x+150-x), 360-(current_y+150-y), relative=True)
            xx=640-(current_x+150-x)
            yy=360-(current_y+150-y)
            print(f"size ---------- {w}")
            xx=xx*(80/w)
            yy=yy*(80/w)
            pydirectinput.moveRel(int(xx), int(yy), relative=True)
            # pyautogui.move(1, 1)
    
    cv2.imshow("www",img)
    # 'q' 키를 누르면 종료
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('p'):
        time.sleep(5)
    
# 사용한 자원 해제
cv2.destroyAllWindows()


0: 320x320 (no detections), 8.8ms
Speed: 1.5ms preprocess, 8.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 5.9ms
Speed: 0.0ms preprocess, 5.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.2ms
Speed: 1.0ms preprocess, 9.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 4.7ms
Speed: 2.5ms preprocess, 4.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 6.7ms
Speed: 1.4ms preprocess, 6.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 4.5ms
Speed: 1.4ms preprocess, 4.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 5.9ms
Speed: 1.2ms preprocess, 5.9ms inference, 1.1ms 

In [6]:
import pyautogui
import time
import pydirectinput
time.sleep(10)
pydirectinput.moveTo(100, 150) # Move the mouse to the x, y coordinates 100, 150.
pydirectinput.click() # Click the mouse at its current location.
pydirectinput.click(200, 220) # Click the mouse at the x, y coordinates 200, 220.
pydirectinput.move(None, 10)  # Move mouse 10 pixels down, that is, move the mouse relative to its current position.
pydirectinput.doubleClick() # Double click the mouse at the
pydirectinput.press('esc') # Simulate pressing the Escape key.
pydirectinput.keyDown('shift')
pydirectinput.keyUp('shift')

True